In [ ]:
### {“tweet content” :{ Timestamp: DDMMYYYY, Twitter account (str), Num of comments/retweets (int):, Comment sentiment (str):, Tweet sentiment (str):, } }

# 1. request (parameters) --> sqs --> tweepy access data
# 2. (select data) data --> dump to s3
# 3. select data to store in dynamodb?
# 4. conduct sentiment analysis on data --> store in dynamodb? // parallelize by sfn
# 5. spark app?


In [1]:
inp = input("Do you already have a tweets dataset to upload (y/n)? If no, let's start scrapping!")
if inp == 'n':
    exit()
keyword = input('What is the topic that you are interested in?')
language = input('What language should the tweets be in?')
before_time = input('What is the time window of your tweets (before yyyy-mm-dd)?')
after_time = input('What is the time window of your tweets (after yyyy-mm-dd)?')
num_tweets = input('How many tweets do you want?')

In [13]:
from datetime import datetime

datetime.strptime(before_time, '%Y/%m/%d ').date()

datetime.datetime(2022, 6, 2, 0, 0)

In [7]:
!pip install tweepy

  Using cached tweepy-4.10.0-py3-none-any.whl (94 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.26.0
    Uninstalling requests-2.26.0:
      Successfully uninstalled requests-2.26.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.


In [20]:
import tweepy
import json

keys = {'consumer_key' : "tr0R2Csjg2rKqWbaTGnvCOLKo",
        'consumer_secret' : "pqd5aVWeypQq3KrUXxUxY5MTJEwUq25unlDApNRTzq78rUT38K",
        'access_token' : "819043815604375552-gp2URG92uuUTVtNTJbOzbKZMkgG1yey",
        'access_token_secret' : "5FQhsXDoSjlnwqo6ortXWbmqvqRWjpmfFc2ueirAYhn4w"}
auth = tweepy.OAuth1UserHandler(consumer_key=keys.get('consumer_key'), 
                                    consumer_secret=keys.get('consumer_secret'), 
                                    access_token=keys.get('access_token'),
                                    access_token_secret=keys.get('access_token_secret'))
api = tweepy.API(auth)
response = api.search_tweets('abortion', lang = 'en', result_type = 'mixed')
# json.dumps(response, default=str)


In [24]:
response[0]._json['created_at']

'Wed Jun 01 12:19:02 +0000 2022'

In [1]:
import boto3
import json
import dataset

sqs = boto3.client('sqs')
aws_lambda = boto3.client('lambda')
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')
dynamodb = boto3.resource('dynamodb')
dynamo_client = boto3.client('dynamodb')
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')
rds = boto3.client('rds')

In [2]:
# Create S3 bucket to store raw JSON data
s3.create_bucket(Bucket='raw-tweet-bucket')

# Create DynamoDB Table for storing processed data
# 'id' will be userID
# try:
#     table = dynamodb.create_table(
#         TableName='tweet_DB',
#         KeySchema=[
#             {
#                 'AttributeName': 'id',
#                 'KeyType': 'HASH'
#             }
#         ],
#         AttributeDefinitions=[
#             {
#                 'AttributeName': 'id',
#                 'AttributeType': 'S'
#             }
#         ],
#         ProvisionedThroughput={
#             'ReadCapacityUnits': 1,
#             'WriteCapacityUnits': 1
#         }    
#     )
# except dynamo_client.exceptions.ResourceInUseException:
#     table = dynamodb.Table('tweet_DB')

# # Wait until AWS confirms that table exists before moving on
# table.meta.client.get_waiter('table_exists').wait(TableName='tweet_DB')

{'ResponseMetadata': {'RequestId': 'WGC0Z0X2BZ84NNFT',
  'HostId': 'TuGHcp0pYLOETviFt9Tyg3I2H7ZAk/L7qMQeIMQzz9ooTG08rlHiBLVAmMiaMY2bdjvEMifDnY8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'TuGHcp0pYLOETviFt9Tyg3I2H7ZAk/L7qMQeIMQzz9ooTG08rlHiBLVAmMiaMY2bdjvEMifDnY8=',
   'x-amz-request-id': 'WGC0Z0X2BZ84NNFT',
   'date': 'Fri, 03 Jun 2022 14:46:23 GMT',
   'location': '/raw-tweet-bucket',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/raw-tweet-bucket'}

In [3]:
try:
    response = rds.create_db_instance(
        DBInstanceIdentifier='relational-db',
        DBName='twitter_sentiment',
        MasterUsername='username',
        MasterUserPassword='password',
        DBInstanceClass='db.t2.micro',
        Engine='MySQL',
        AllocatedStorage=5
    )
except:
    pass

# Wait until DB is available to continue
rds.get_waiter('db_instance_available').wait(DBInstanceIdentifier='relational-db')

# Describe where DB is available and on what port
db = rds.describe_db_instances()['DBInstances'][0]
ENDPOINT = db['Endpoint']['Address']
PORT = db['Endpoint']['Port']
DBID = db['DBInstanceIdentifier']

print(DBID,
      "is available at", ENDPOINT,
      "on Port", PORT,
     )   

relational-db is available at relational-db.ccps3ediik0q.us-east-1.rds.amazonaws.com on Port 3306


In [4]:
# Get Name of Security Group
SGNAME = db['VpcSecurityGroups'][0]['VpcSecurityGroupId']

# Adjust Permissions for that security group so that we can access it on Port 3306
# If already SG is already adjusted, print this out
try:
    ec2 = boto3.client('ec2')
    data = ec2.authorize_security_group_ingress(
            GroupId=SGNAME,
            IpPermissions=[
                {'IpProtocol': 'tcp',
                 'FromPort': PORT,
                 'ToPort': PORT,
                 'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
            ]
    )
except ec2.exceptions.ClientError as e:
    if e.response["Error"]["Code"] == 'InvalidPermission.Duplicate':
        print("Permissions already adjusted.")
    else:
        print(e)

Permissions already adjusted.


In [5]:
# connect to RDS
db_url = 'mysql+mysqlconnector://{}:{}@{}:{}/twitter_sentiment'.format('username', 'password', ENDPOINT, PORT)
db = dataset.connect(db_url)

ModuleNotFoundError: No module named 'mysql'

In [8]:
# Create Lambda Function
with open('./deployment.zip', 'rb') as f:
    lambda_zip = f.read()

try:
    # If function hasn't yet been created, create it
    response = aws_lambda.create_function(
        FunctionName='tweet_lambda',
        Runtime='python3.9',
        Role=role['Role']['Arn'],
        Handler='lambda_function.lambda_handler',
        Code=dict(ZipFile=lambda_zip),
        Timeout=3
    )
except aws_lambda.exceptions.ResourceConflictException:
    # If function already exists, update it based on zip file contents
    response = aws_lambda.update_function_code(
    FunctionName='tweet_lambda',
    ZipFile=lambda_zip
    )

lambda_arn = response['FunctionArn']  

# Create SQS Queue
try:
    queue_url = sqs.create_queue(QueueName='Tweet')['QueueUrl']
except sqs.exceptions.QueueNameExists:
    queue_url = [url
                 for url in sqs.list_queues()['QueueUrls']
                 if 'Tweet' in url][0]
    
sqs_info = sqs.get_queue_attributes(QueueUrl=queue_url,
                                    AttributeNames=['QueueArn'])
sqs_arn = sqs_info['Attributes']['QueueArn']

# Trigger Lambda Function when new data enter SQS Queue
try:
    response = aws_lambda.create_event_source_mapping(
        EventSourceArn=sqs_arn,
        FunctionName='tweet_lambda',
        Enabled=True,
        BatchSize=10
    )
except aws_lambda.exceptions.ResourceConflictException:
    es_id = aws_lambda.list_event_source_mappings(
        EventSourceArn=sqs_arn,
        FunctionName='tweet_lambda'
    )['EventSourceMappings'][0]['UUID']
    
    response = aws_lambda.update_event_source_mapping(
        UUID=es_id,
        FunctionName='tweet_lambda',
        Enabled=True,
        BatchSize=10
    )

# get data about table (should currently be no items in table)
print("SQS -> Lambda Architecture has been Launched")

SQS -> Lambda Architecture has been Launched


In [9]:
with open('twitter_sentiment_deployment_package.zip', 'rb') as f:
    lambda_zip = f.read()

try:
    # If function hasn't yet been created, create it
    response = aws_lambda.create_function(
        FunctionName='twitter_sentiment',
        Runtime='python3.9',
        Role=role['Role']['Arn'],
        Handler='lambda_function.lambda_handler',
        Code=dict(ZipFile=lambda_zip),
        Timeout=100
    )
except aws_lambda.exceptions.ResourceConflictException:
    # If function already exists, update it based on zip
    # file contents
    response = aws_lambda.update_function_code(
    FunctionName='twitter_sentiment',
    ZipFile=lambda_zip
    )

lambda_arn = response['FunctionArn']    

In [10]:
!python sfn_setup.py

In [11]:
def send_request(tweet_data, sqs_url):
    response = sqs.send_message(QueueUrl=sqs_url,
                                MessageBody=json.dumps(tweet_data))
                                
    return response['ResponseMetadata']['HTTPStatusCode']
    # return response

In [12]:
!python twint_search.py

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs
^C
Traceback (most recent call last):
  File "/opt/anaconda3/envs/textanalysis/lib/python3.9/site-packages/twint/run.py", line 65, in Feed
    self.feed, self.init = feed.Json(response)
  File "/opt/anaconda3/envs/textanalysis/lib/python3.9/site-packages/twint/feed.py", line 42, in Json
    json_response = loads(response)
  File "/opt/anaconda3/envs/textanalysis/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/anaconda3/envs/textanalysis/lib/python3.9/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/anaconda3/envs/textanalysis/lib/python3.9/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecod

In [13]:
param = {
    "keyword": "abortion",
}
data = [{'tweet_id': '0001',
        'TimeStamp': '2022-06-02 18:49:35 UTC', #format slightly different: '2022-06-02 18:49:35 UTC'
        'Twitter account': '0001',
        'Num of comments/retweets': 10,
        'Likes': 10,
        'Reply_to': '0002',
        'text': 'abortion'}, {
        'tweet_id': '0002',
        'TimeStamp': '2022-06-02 18:49:35 UTC', #format slightly different: '2022-06-02 18:49:35 UTC'
        'Twitter account': '0002',
        'Num of comments/retweets': 10,
        'Likes': 10,
        'Reply_to': '0003', 
        'text': 'abortion'}]
send_request(data, sqs_url=queue_url)

200

In [15]:
tweet_batches = [{'batch': []} for i in range(10)]
batch_size = int(len(response)/10)
remaining = len(response)%10
batch_num = 0

for r in response:
    tweet_dict = {
                'id': r._json['id'],
                'TimeStamp': r._json['created_at'],
                'Twitter account': r._json['user']['id_str'],
                'Num of comments/retweets': r._json['retweet_count'],
                'Likes': r._json['favorite_count'],
                'Reply_to': r._json['in_reply_to_user_id']
            }
    tweet_batches[batch_num]['batch'].append(tweet_dict)
    if len(tweet_batches[batch_num]['batch']) == batch_size:
        if remaining > 0:
            remaining -=1 
            continue
    batch_num += 1

tweet_batches

[{'batch': [{'id': 1531674365242359808,
    'TimeStamp': 'Tue May 31 16:30:00 +0000 2022',
    'Twitter account': '815733290955112448',
    'Num of comments/retweets': 5326,
    'Likes': 27356,
    'Reply_to': None},
   {'id': 1531723932767617024,
    'TimeStamp': 'Tue May 31 19:46:58 +0000 2022',
    'Twitter account': '878284831',
    'Num of comments/retweets': 3905,
    'Likes': 13427,
    'Reply_to': None}]},
 {'batch': [{'id': 1531621877726208010,
    'TimeStamp': 'Tue May 31 13:01:26 +0000 2022',
    'Twitter account': '19706851',
    'Num of comments/retweets': 1023,
    'Likes': 2077,
    'Reply_to': None},
   {'id': 1532088007917981696,
    'TimeStamp': 'Wed Jun 01 19:53:40 +0000 2022',
    'Twitter account': '1311079340168404997',
    'Num of comments/retweets': 8,
    'Likes': 0,
    'Reply_to': None}]},
 {'batch': [{'id': 1532088005065945091,
    'TimeStamp': 'Wed Jun 01 19:53:39 +0000 2022',
    'Twitter account': '1224132582717251585',
    'Num of comments/retweets': 532

In [14]:
batch_size

1

In [11]:
db['tweets_table'].columns

['id',
 'tweet_id',
 'sentiment',
 'sentiment_score',
 'timestamp',
 'user_id',
 'num_retweets',
 'num_likes',
 'in_reply_to',
 'text']

In [12]:
list(db['tweets_table'].find(id=10))

[OrderedDict([('id', 10),
              ('tweet_id', 1532545194909908999),
              ('sentiment', 'NEUTRAL'),
              ('sentiment_score',
               {'Mixed': 0.0004169417079538107,
                'Neutral': 0.5684710144996643,
                'Negative': 0.40361034870147705,
                'Positive': 0.02750173397362232}),
              ('timestamp', '2022-06-03 02:10:22'),
              ('user_id', '42301087'),
              ('num_retweets', 31),
              ('num_likes', 0),
              ('in_reply_to', None),
              ('text',
               'RT @MarkLevineNYC: BIG: NY just passed major package of gun safety bills...\n* Raise age to buy semiautomatic rifle to 21, &amp; require permit…')])]

In [14]:
all_data = db['tweets_table'].all()
all_data = [data for data in all_data]
with open('rds.json', "w") as outfile:
    json.dump(all_data, outfile)

In [16]:
json.load(open('rds.json'))

[{'id': 1,
  'tweet_id': 1532545093835571206,
  'sentiment': 'NEUTRAL',
  'sentiment_score': {'Mixed': 0.0005819596699438989,
   'Neutral': 0.8748773336410522,
   'Negative': 0.12291298806667328,
   'Positive': 0.0016277596587315202},
  'timestamp': '2022-06-03 02:09:58',
  'user_id': '271045715',
  'num_retweets': 0,
  'num_likes': 0,
  'in_reply_to': 1349149096909668363,
  'text': '@POTUS I agree sir. So you’ve decided that abortion is illegal ?'},
 {'id': 2,
  'tweet_id': 1532544485317558273,
  'sentiment': 'NEUTRAL',
  'sentiment_score': {'Mixed': 0.00025615698541514575,
   'Neutral': 0.8686162233352661,
   'Negative': 0.1032906025648117,
   'Positive': 0.027836980298161507},
  'timestamp': '2022-06-03 02:07:33',
  'user_id': '633642303',
  'num_retweets': 26,
  'num_likes': 0,
  'in_reply_to': None,
  'text': 'RT @luckydeviIgaia: The majority of white ppl in the midterms will vote GOP. The question is will enough of them vote for Dems to do what n…'},
 {'id': 3,
  'tweet_id': 1532